In [1]:
# set seed for reproducibility purposes.
from numpy.random import seed
seed(888)
from tensorflow.random import set_seed
set_seed(404)

In [2]:
import tensorflow as tf
import numpy as np
import os

In [3]:
# CONSTANTS
X_TRAIN_PATH = 'MNIST_Dataset/digit_xtrain.csv'
X_TEST_PATH = 'MNIST_Dataset/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST_Dataset/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST_Dataset/digit_ytest.csv'

## *Load Data*

In [4]:
%%time

y_train_data = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
print('Y Train shape: ', y_train_data.shape)

y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
print('Y Test shape: ', y_test.shape)

Y Train shape:  (60000,)
Y Test shape:  (10000,)
CPU times: user 258 ms, sys: 349 µs, total: 258 ms
Wall time: 259 ms


In [5]:
%%time

x_train_data = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
print('X Train shape: ', x_train_data.shape)

x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
print('X Test shape: ', x_test.shape)

X Train shape:  (60000, 784)
X Test shape:  (10000, 784)
CPU times: user 28.6 s, sys: 1.55 s, total: 30.1 s
Wall time: 30.2 s


---